In [1]:
from konlpy.tag import Komoran
import re
import pandas as pd
from gensim import models
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import os

tqdm.pandas()

In [3]:
# add user-defined dictionary
komoran = Komoran(userdic="../vocab/user_dic.txt")

def tokenize(text):
    pos_tag = ["NNG", "NNP", "NNB", "NP", "NR", "VA", "MM", "MAG"]
    tokens_pos = komoran.pos(text)
    tokens_word = []

    for tag in tqdm(tokens_pos):
        if tag[1] in pos_tag:
            tokens_word.append(tag[0])

    return re.sub("\.", "", " ".join(tokens_word))

In [4]:
df = pd.read_csv("../database/review_pseudolabeled.csv")
df[:5]

,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user,sentence,fixed,label,probability
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028,영등포에서는 이곳만 와요 너무 좋습니다,영등포에서는 이곳만 와요,1,97.836401
1,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028,영등포에서는 이곳만 와요 너무 좋습니다,너무 좋습니다,1,99.999919
2,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,체크인 시간이 너무너무 깁니다,0,91.076837
3,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,빨리할 수 있는 방법을 구상해야 할 것 같습니다,0,56.304458
4,609182b3f5c0adc042e03b0a,5,잘 머물다 갑니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12752105,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,5,그때그시절처렁,잘 머물다 갑니다,잘 머물다 갑니다,1,99.999904


In [6]:
dataset = []
for sent in tqdm(df["sentence"]):
    dataset.append(tokenize(sent).split())

In [7]:
w2v = models.Word2Vec(dataset, sg=1, window=8, min_count=1)
w2v.init_sims(replace=True)

In [8]:
def get_keywords(file):
    df = pd.read_csv(file)
    keywords = df.iloc[:30, 0]
    keywords = keywords.values.tolist()

    return keywords

In [9]:
def make_group(keywords):
    groups = []
    for word in tqdm(keywords):
        sim = []
        for i in w2v.wv.most_similar(word, topn=20):
            sim.append(i[0])
        group_words = [word]
        keywords.remove(word)
        for i in keywords:
            if i in sim:
                group_words.append(i)
                keywords.remove(i)
    groups.append(group_words)

    return groups

In [10]:
def identity_tokenize(text):
    return text

In [11]:
def get_hotel_name(path):
    name = path[5:]
    name = name[:-4]

    return name

In [12]:
def get_rep(groups):
    tfidfv = TfidfVectorizer(tokenizer=identity_tokenize,lowercase=False)    
    X = tfidfv.fit_transform(dataset)
    tfidf = X.toarray()

    rep = []

    for i in tqdm(range(len(groups))):
        if len(groups[i]) > 1:
            ar = [0] * len(groups)
            for j in range(len(groups[i])):
                c2 = 0
                for c in dataset:
                    for c1 in groups[i]:
                        if c1 in c:
                            c2 += 1
                            break

                z = tfidfv.vocabulary_[groups[i][j]]
                t = 0

                for j1 in range(len(tfidf)):
                    t += tfidf[j1][z]
                t = t / c2
                t1 = 0

                for j1 in range(len(groups[i])):
                    if j1 != j:
                        t += w2v.wv.similarity(w1=groups[i][j], w2=groups[i][j1])

                t1 = t1 / (len(groups[i]) - 1)
                ar[j] = t + t1
            rep.append(groups[i][ar.index(max(ar))])
        else:
            rep.append(groups[i][0])

    return rep

In [13]:
path = "../vocab/key"

hotel_keyword_list = []
hotels = os.listdir(path)

for hotel in tqdm(hotels):
    keywords = get_keywords(os.path.join(path, hotel))
    groups = make_group(keywords)
    rep = get_rep(groups)

    for i in range(len(rep)):
        hotel_keyword_list.append([get_hotel_name(hotel), rep[i], groups[i]])

hotel_keyword = pd.DataFrame(hotel_keyword_list, columns=["Hotel", "Rep", "Keywords"])
hotel_keyword.to_csv("hotel_keyword.csv")